In [36]:
from peft import (
    get_peft_model,
    LoraConfig,
    prepare_model_for_int8_training,
    PeftModelForCausalLM
)
import torch

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer


tokenizer = LlamaTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")
original_llama = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [41]:
pm = PeftModelForCausalLM.from_pretrained(original_llama, '/home/jovyan/llama/peft_trained')

In [27]:
tokenizer.encode("\n", add_special_tokens=False)

[29871, 13]

In [28]:
tokenizer.decode([13])

'\n'

In [29]:
LORA_CONFIG = {
    "r": 12,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "bias": "none",
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
    "task_type": "CAUSAL_LM",
}

In [30]:
lora_config = LoraConfig(**LORA_CONFIG)
# model = prepare_model_for_int8_training(model, use_gradient_checkpointing=False)
model = get_peft_model(model, lora_config)

In [31]:
lora_weights = torch.load("trained/checkpoint-8000/merged_model.pt")

In [32]:
model.load_state_dict(lora_weights)

<All keys matched successfully>

In [35]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=12, bias=False)
                (lora_B): Linear(in_features=12, out_features=4096, bias=False)
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=12, bias=False)
                (lora_B): Linear(in_features=12, out_features=4096, bias=False)
              )
              (v_proj): Linear(
            

In [34]:
model.save_pretrained("peft_trained")

## Fine-tuned llama

In [33]:
inp = """Это правило не везде, однако, могло быть проведено с безусловной последовательностью; практика вызывала частные отклонения, создавала те или иные комбинации. Если известное божество считалось главным божеством известного государства, то государство признавало иногда (как в Афинах) вместе с тем и некоторые другие культы; наряду с этими общегосударственными культами существовали и отдельные культы государственных делений (например, афинских демов), и культы частноправового значения (например, домашние или семейные), а также культы частных обществ или лиц.
Вопрос: Везде ли правило могло быть проведено с безусловной последовательностью?
Ответ (да/нет):"""

In [31]:
inputs = tokenizer(inp, return_tensors="pt")

In [32]:
output = model.generate(input_ids=inputs.input_ids, 
                        max_new_tokens=100,
                        temperature=0.98,
                        repetition_penalty=1.2)

In [33]:
print("sdf")

sdf


In [34]:
out = tokenizer.batch_decode(output, clean_up_tokenization_spaces=False)

In [35]:
print(out[0])

<s>По данным экспертов, в Ялте по итогам 25 апреля текущего года по сравнению с предыдущим годом аренда жилья в среднем подешевела на 35 процентов, до 39,4 тысячи рублей в месяц. Вторым в списке курортных городов с наиболее подешевевшей арендой специалисты назвали Калининград со снижением на 16 процентов, до 19,4 тысячи рублей в месяц. Замкнул тройку Севастополь с отрицательной динамикой - за год цены выросли почти на 80 процентов, до 70,5 тысяч рублей в месяц.
Всего в рейтинге "Коммерсанта" вошло 20 крупнейших российских курортов: Сочи (№ 1), Анапа (№ 2), Новороссийск (№ 3), Геленджик (№ 4


## ruGpt3large

In [42]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

gpt_model_name_or_path = "ai-forever/rugpt3large_based_on_gpt2"
gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name_or_path)


In [43]:
gpt_tokenizer("По данным экспертов, в Ялте по итогам 25 апреля текущего года по сравнению с предыдущим годом аренда жилья в среднем подешевела на 35 процентов")

{'input_ids': [4408, 3761, 14971, 16, 282, 34471, 464, 334, 8872, 2706, 4238, 16642, 779, 334, 7413, 281, 49509, 12260, 25581, 12712, 282, 11099, 46698, 555, 309, 6603, 7540], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name_or_path).cpu()

In [37]:
input_ids = gpt_tokenizer.encode(inp, return_tensors="pt").cpu()
out = gpt_model.generate(input_ids.cpu(), 
                         max_new_tokens=100,
                         temperature=0.98,
                         repetition_penalty=1.7)

generated_text = list(map(gpt_tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


По данным экспертов, в Ялте по итогам 25 апреля текущего года по сравнению с предыдущим годом аренда жилья в среднем подешевела на 35 процентов, до 39,4 тысячи рублей в месяц. Вторым в списке курортных городов с наиболее подешевевшей арендой специалисты назвали Калининград со снижением на 16 процентов, до 19,4 тысячи рублей в месяц. Замкнул тройку Севастополь с отрицательной динамикой - минус 15 процентных пунктов и 13 тысяч за квадратный метр жилой площади соответственно (в Симферополе жилье подорожало всего лишь чуть более чем вдвое).<s>
В Москве задержали мужчину из-за нападения грабителя банка Полиция задержала подозреваемого во взяточничестве жителя Москвы после того как он попытался ограбить банк «Солидарность». Об этом сообщает РИА Новости. По словам представителя пресс службы столичного главка МВД Андрея Галиакберова, задержанный был задержан при попытке ограбления отделения Сбербанка России №


## Original llama

In [34]:
inputs = tokenizer(inp, return_tensors="pt")

In [35]:
output = original_llama.generate(input_ids=inputs.input_ids, 
                                 max_new_tokens=2)

In [36]:
out = tokenizer.batch_decode(output, clean_up_tokenization_spaces=False)

In [37]:
print(out[0])

<s>Это правило не везде, однако, могло быть проведено с безусловной последовательностью; практика вызывала частные отклонения, создавала те или иные комбинации. Если известное божество считалось главным божеством известного государства, то государство признавало иногда (как в Афинах) вместе с тем и некоторые другие культы; наряду с этими общегосударственными культами существовали и отдельные культы государственных делений (например, афинских демов), и культы частноправового значения (например, домашние или семейные), а также культы частных обществ или лиц.
Вопрос: Везде ли правило могло быть проведено с безусловной последовательностью?
Ответ (да/нет): Нет
